In [65]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as px
import os

In [131]:
path = kagglehub.dataset_download("hopesb/student-depression-dataset")
data_dir=r"C:\Users\BIT\.kaggle\archive"
train_csv= data_dir + r'\Student Depression Dataset.csv'
raw_df = pd.read_csv(train_csv)

In [119]:
raw_df.drop(columns=['Work Pressure', 'Job Satisfaction'], errors='ignore', inplace=True)
raw_df = raw_df.loc[raw_df['Profession'] == 'Student']
raw_df['Financial Stress']=raw_df['Financial Stress'].fillna(raw_df['Financial Stress'].median())
from sklearn.model_selection import train_test_split
train_val_df,test_df=train_test_split(raw_df,test_size=0.2,random_state=42)
train_df,val_df=train_test_split(train_val_df,test_size=0.25,random_state=42)

In [120]:
input_col=list(train_df.columns)[1:-1]
target_col='Depression'

In [121]:
train_input=train_df[input_col].copy()
train_target=train_df[target_col].copy()
val_input=val_df[input_col].copy()
val_target=val_df[target_col].copy()
test_input=test_df[input_col].copy()
test_target=test_df[target_col].copy()

In [122]:
import numpy as np
numeric_col=train_input.select_dtypes(include=np.number).columns.tolist()
catagorical_col=train_input.select_dtypes('object').columns.tolist()
train_input[catagorical_col].nunique ()
numeric_coll = raw_df.select_dtypes(include=['float64', 'int64']).columns

In [123]:
from sklearn.impute import SimpleImputer 
imputer= SimpleImputer(strategy='mean')
imputer.fit(raw_df[numeric_col])
train_input[numeric_col]=imputer.transform(train_input[numeric_col])
val_input[numeric_col]=imputer.transform(val_input[numeric_col])
test_input[numeric_col]=imputer.transform(test_input[numeric_col])

In [124]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
scaler.fit(raw_df[numeric_col])
train_input[numeric_col]=scaler.transform(train_input[numeric_col])
val_input[numeric_col]=scaler.transform(val_input[numeric_col])
test_input[numeric_col]=scaler.transform(test_input[numeric_col])
train_input[numeric_col].describe()

,Age,Academic Pressure,CGPA,Study Satisfaction,Work/Study Hours,Financial Stress
count,16722.000000,16722.000000,16722.000000,16722.000000,16722.000000,16722.000000
mean,0.190236,0.626050,0.765194,0.589044,0.594735,0.535477
std,0.118860,0.275535,0.147253,0.272073,0.310727,0.358802
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.073171,0.400000,0.628000,0.400000,0.333333,0.250000
50%,0.170732,0.600000,0.777000,0.600000,0.666667,0.500000
75%,0.292683,0.800000,0.892000,0.800000,0.833333,0.750000
max,0.926829,1.000000,1.000000,1.000000,1.000000,1.000000


In [130]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
encoder.fit(raw_df[catagorical_col])
raw_df=raw_df[catagorical_col].fillna('unknown')
encoded_cols= list(encoder.get_feature_names_out(catagorical_col))
train_input[encoded_cols]=encoder.transform(train_input[catagorical_col])
val_input[encoded_cols]=encoder.transform(val_input[catagorical_col])
test_input[encoded_cols]=encoder.transform(test_input[catagorical_col])
x_train=train_input[numeric_col + encoded_cols]
x_val=val_input[numeric_col + encoded_cols]
x_test=test_input[numeric_col + encoded_cols]

In [136]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,train_target)
pre=model.predict(x_train)  
pre

array([0, 1, 0, ..., 1, 0, 0])

In [137]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(train_target,pre)

0.8491807200095682